In [1]:
# pip install neo4j
from neo4j import GraphDatabase

In [ ]:
# First provide your neo4j account information for connection
username = input("Please input your neo4j username (neo4j is default): ")
password = input("Please input your password: ")

In [7]:
# Connect to the neo4j database server
graphdb = GraphDatabase.driver(uri = "bolt://localhost:7687", auth = (username, password))

In [38]:
session = graphdb.session()

In [44]:
# Input your name
myname = input("What is your name?")

What is your name?Ivan Wang


In [45]:
# Create your user node to the database
q1 = 'CREATE (p:User {{name:"{}"}})'.format(myname)
session.run(q1)

In [49]:
# Input the number of movies you want to rate. At least 5 is required in order to get enough information to get recommendations
n = int(input("How many movies you want to rate?(At least 5) "))

# Input movvies' titles and ratings
movie_list = dict()
for i in range(n):
    data = input('Please enter title whose intial letter is capitalized & rating separated by ":"')
    temp = data.split(':')
    movie_list[temp[0]] = int(temp[1])

# Statement to create a rating relationship between users and movies
q2 = """MATCH (p:User),(m:Movie)
WHERE p.name = '{0}' AND m.title = '{1}'
CREATE (p)-[r:RATED {{rating:{2}}}]->(m)
RETURN type(r)"""

# Establish a rating relationship
for key, value in movie_list.items():
	session.run(q2.format(myname, key, value))

How many movies you want to rate?(At least 10) 4
Please enter name & rating separated by ":" Toy Story:8
Please enter name & rating separated by ":" Heat:6
Please enter name & rating separated by ":" Copycat:8
Please enter name & rating separated by ":" Powder:2


In [71]:
# This simple approach is to only take genres into consideration
# Based on the movies rated by you, what genres have a rating greater than the average rating. Output similar movies as recommendations
# It only focuses on yourself.
q3 = """MATCH (u:User {{name: "{0}"}})-[r:RATED]->(m:Movie)
WITH u, avg(r.rating) AS mean

MATCH (u)-[r:RATED]->(m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE r.rating > mean

WITH u, g, COUNT(*) AS score

MATCH (g)<-[:IN_GENRE]-(rec:Movie)
WHERE NOT EXISTS((u)-[:RATED]->(rec))

RETURN rec.title AS recommendation, rec.year AS year, COLLECT(DISTINCT g.name) AS genres, SUM(score) AS sscore
ORDER BY sscore DESC LIMIT 10""".format(myname)
recommendations1_temp = session.run(q3)
recommendations1 = []
for n in recommendations1_temp.values():
    movie = n[0]
    recommendations1.append(movie)
print(recommendations1)

['Motorama', 'Rubber', 'Who Framed Roger Rabbit?', 'Seven-Per-Cent Solution, The', 'Wonderful World of the Brothers Grimm, The', 'Interstate 60', 'Inside Out', 'Pulse', 'Invisible, The', "Kiki's Delivery Service (Majo no takkyûbin)"]


In [64]:
# This approach is using cosine similarity to find users who have similar preferences.
# The higher sim is, the more similar preference a user has
# It will show the user who has the most similar preference for watching movies
# Then we decide to choose 10 movies which have the highest rating from the user as recommendation

q4 = """MATCH (u1:User {{name: "{}"}})-[x:RATED]->(m:Movie)<-[y:RATED]-(u2:User)
WITH COUNT(m) AS numbermovies, SUM(x.rating * y.rating) AS xyDotProduct,
SQRT(REDUCE(xDot = 0.0, a IN COLLECT(x.rating) | xDot + a^2)) AS xLength,
SQRT(REDUCE(yDot = 0.0, b IN COLLECT(y.rating) | yDot + b^2)) AS yLength,
u1, u2 WHERE numbermovies > 3
RETURN u1.name, u2.name, xyDotProduct / (xLength * yLength) AS sim
ORDER BY sim DESC
LIMIT 1;""".format(myname)

result = session.run(q4)

# Save the user name
user_name = result.values()[0][1]

# Display the 10 highest rating movies of the user
q5 = """MATCH (u:User {{name: "{0}"}})-[x:RATED]->(m:Movie)
RETURN m.title
ORDER BY x.rating DESC
LIMIT 10""".format(user_name)

recommendations2_temp = session.run(q5)
recommendations2 = recommendations2_temp.values()
print(recommendations2)

[["Who's Afraid of Virginia Woolf?"], ['West Side Story'], ['Player, The'], ['Few Good Men, A'], ['Happiness'], ['And the Band Played On'], ['Pleasantville'], ['Life Is Beautiful (La Vita è bella)'], ["Ferris Bueller's Day Off"], ['Midnight Cowboy']]
